In [1]:
#I TRRIED WITH FIXING THE OUTLIERS BUT ALSO HERE AS THE CLUSTERING THE 
#SITUATION WORSE FIXING OUTLIERS AS MEDIAN. ANYWAY I THINK WE HAVE AN
#INSUFFICIENT NUMBER OF RULES. MAYBE THERE IS A PROBLEM SOMEWHERE

%matplotlib inline
import math
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import mode
from scipy.spatial.distance import pdist
from scipy.cluster.hierarchy import linkage, dendrogram
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score


from collections import defaultdict
from scipy.stats.stats import pearsonr
from sklearn.cluster import DBSCAN
from scipy.spatial.distance import pdist, squareform
from scipy.stats import mode
from scipy.spatial.distance import pdist
from scipy.cluster.hierarchy import linkage, dendrogram

In [2]:
df = pd.read_csv('training.csv')

In [3]:
df['MMRAcquisitionAuctionAveragePrice'].fillna(df['MMRAcquisitionAuctionAveragePrice'].median(), inplace = True)
df['MMRAcquisitionAuctionCleanPrice'].fillna(df['MMRAcquisitionAuctionCleanPrice'].median(), inplace = True)
df['MMRAcquisitionRetailAveragePrice'].fillna(df['MMRAcquisitionRetailAveragePrice'].median(), inplace = True)
df['MMRAcquisitonRetailCleanPrice'].fillna(df['MMRAcquisitonRetailCleanPrice'].median(), inplace = True)
df['MMRCurrentAuctionAveragePrice'].fillna(df['MMRCurrentAuctionAveragePrice'].median(), inplace = True)
df['MMRCurrentAuctionCleanPrice'].fillna(df['MMRCurrentAuctionCleanPrice'].median(), inplace = True)
df['MMRCurrentRetailAveragePrice'].fillna(df['MMRCurrentRetailAveragePrice'].median(), inplace = True)
df['MMRCurrentRetailCleanPrice'].fillna(df['MMRCurrentRetailCleanPrice'].median(), inplace = True)

In [4]:
df['MMR_factor'] = ( df['MMRAcquisitionAuctionAveragePrice'] + df['MMRAcquisitonRetailCleanPrice'] + df['MMRCurrentAuctionCleanPrice'] + df['MMRCurrentRetailAveragePrice']) / 4

In [5]:
#BINNING NUMERICAL VALUES
#We have to use the sturges' rule but that gives us 17 bin but i think 10 is better because we obtain more rules
df['VehBCost_bin'] = pd.cut(df.VehBCost.astype(int), 10, right = False) 
df['VehOdo_bin'] = pd.cut(df.VehOdo.astype(int), 10, right = False)
df['WarrantyCost_bin'] = pd.cut(df.WarrantyCost.astype(int), 10, right = False)

In [6]:
#SELECT THE COLUMNS I AM INTERESTED IN
df_new = df[['Auction','VehicleAge','Make','Trim','Color','WheelType','Nationality','Size','VNST','VehBCost_bin','VehOdo_bin','WarrantyCost_bin']]

In [7]:
#FIX MISSING VALUES
df_new.isnull().sum()

Auction                0
VehicleAge             0
Make                   0
Trim                1911
Color                  7
WheelType           2577
Nationality            4
Size                   4
VNST                   0
VehBCost_bin           0
VehOdo_bin             0
WarrantyCost_bin       0
dtype: int64

In [8]:
df_new.dropna(subset = ['Color','Nationality','Size'], inplace=True)

C:\Users\bonin\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [9]:
df_new.Trim.fillna('Bas', inplace = True, limit=801)

C:\Users\bonin\Anaconda3\lib\site-packages\pandas\core\generic.py:5434: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [10]:
df_new.Trim.fillna('LS', inplace = True, limit=580)

In [11]:
df_new.Trim.fillna('SE', inplace = True, limit=530)

In [12]:
df_new.WheelType.fillna('Alloy', inplace = True, limit=1377)

In [13]:
df_new.WheelType.fillna('Covers', inplace = True, limit=1200)

In [14]:
df_new.isnull().sum()

Auction             0
VehicleAge          0
Make                0
Trim                0
Color               0
WheelType           0
Nationality         0
Size                0
VNST                0
VehBCost_bin        0
VehOdo_bin          0
WarrantyCost_bin    0
dtype: int64

In [15]:
df_new.head()

,Auction,VehicleAge,Make,Trim,Color,WheelType,Nationality,Size,VNST,VehBCost_bin,VehOdo_bin,WarrantyCost_bin
0,ADESA,2,KIA,EX,BLUE,Covers,OTHER ASIAN,MEDIUM,FL,"[3649.4, 7297.8)","[60271.0, 71360.2)","[462.0, 1165.6)"
1,MANHEIM,3,DODGE,SXT,SILVER,Covers,AMERICAN,MEDIUM,FL,"[3649.4, 7297.8)","[82449.4, 93538.6)","[1165.6, 1869.2)"
2,OTHER,2,DODGE,SE,BLACK,Covers,AMERICAN,MEDIUM,OK,"[7297.8, 10946.2)","[38092.6, 49181.8)","[462.0, 1165.6)"
3,OTHER,4,FORD,S,BROWN,Covers,AMERICAN,VAN,AZ,"[3649.4, 7297.8)","[71360.2, 82449.4)","[1165.6, 1869.2)"
4,MANHEIM,4,CHRYSLER,Bas,BLUE,Alloy,AMERICAN,VAN,TX,"[3649.4, 7297.8)","[71360.2, 82449.4)","[1165.6, 1869.2)"


In [16]:
#this code is necessary because if we don't add a string to the algorithm value..
#..when we run the algorithm it doesn't find the attribute values
for i in df_new:
        df_new[i] = df_new[i].astype(str) + "_{}".format(i)

C:\Users\bonin\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [17]:
df_new.head()

,Auction,VehicleAge,Make,Trim,Color,WheelType,Nationality,Size,VNST,VehBCost_bin,VehOdo_bin,WarrantyCost_bin
0,ADESA_Auction,2_VehicleAge,KIA_Make,EX_Trim,BLUE_Color,Covers_WheelType,OTHER ASIAN_Nationality,MEDIUM_Size,FL_VNST,"[3649.4, 7297.8)_VehBCost_bin","[60271.0, 71360.2)_VehOdo_bin","[462.0, 1165.6)_WarrantyCost_bin"
1,MANHEIM_Auction,3_VehicleAge,DODGE_Make,SXT_Trim,SILVER_Color,Covers_WheelType,AMERICAN_Nationality,MEDIUM_Size,FL_VNST,"[3649.4, 7297.8)_VehBCost_bin","[82449.4, 93538.6)_VehOdo_bin","[1165.6, 1869.2)_WarrantyCost_bin"
2,OTHER_Auction,2_VehicleAge,DODGE_Make,SE_Trim,BLACK_Color,Covers_WheelType,AMERICAN_Nationality,MEDIUM_Size,OK_VNST,"[7297.8, 10946.2)_VehBCost_bin","[38092.6, 49181.8)_VehOdo_bin","[462.0, 1165.6)_WarrantyCost_bin"
3,OTHER_Auction,4_VehicleAge,FORD_Make,S_Trim,BROWN_Color,Covers_WheelType,AMERICAN_Nationality,VAN_Size,AZ_VNST,"[3649.4, 7297.8)_VehBCost_bin","[71360.2, 82449.4)_VehOdo_bin","[1165.6, 1869.2)_WarrantyCost_bin"
4,MANHEIM_Auction,4_VehicleAge,CHRYSLER_Make,Bas_Trim,BLUE_Color,Alloy_WheelType,AMERICAN_Nationality,VAN_Size,TX_VNST,"[3649.4, 7297.8)_VehBCost_bin","[71360.2, 82449.4)_VehOdo_bin","[1165.6, 1869.2)_WarrantyCost_bin"


In [18]:
#we have to pass the values of df_new as a list to run in the right way apriori algorithm
baskets = df_new.values.tolist()

In [19]:
#BASKET EXAMPLE
baskets[0]

['ADESA_Auction',
 '2_VehicleAge',
 'KIA_Make',
 'EX_Trim',
 'BLUE_Color',
 'Covers_WheelType',
 'OTHER ASIAN_Nationality',
 'MEDIUM_Size',
 'FL_VNST',
 '[3649.4, 7297.8)_VehBCost_bin',
 '[60271.0, 71360.2)_VehOdo_bin',
 '[462.0, 1165.6)_WarrantyCost_bin']

In [20]:
from fim import apriori

In [21]:
#This is the explaination of the parameters of apriori
print(apriori.__doc__)

apriori (tracts, target='s', supp=10, zmin=1, zmax=None, report='a',
         eval='x', agg='x', thresh=10, prune=None, algo='b', mode='',
         border=None)
Find frequent item sets with the Apriori algorithm.
tracts  transaction database to mine (mandatory)
        The database must be an iterable of transactions;
        each transaction must be an iterable of items;
        each item must be a hashable object.
        If the database is a dictionary, the transactions are
        the keys, the values their (integer) multiplicities.
target  type of frequent item sets to find     (default: s)
        s/a   sets/all   all     frequent item sets
        c     closed     closed  frequent item sets
        m     maximal    maximal frequent item sets
        g     gens       generators
        r     rules      association rules
supp    minimum support of an item set         (default: 10)
        (positive: percentage, negative: absolute number)
conf    minimum confidence of an assoc. rul

In [22]:
rules = apriori(baskets, supp=10, zmin=2, target='r', conf=80, report='YXlC') 
print(len(rules))
contare = 0
for y in rules:
        print('%s -- %s' % (str(contare), y))
        contare = contare + 1

109
0 -- ('AMERICAN_Nationality', ('BLACK_Color',), 83.58543897216273, 10.391434689507495, 0.9701609480623545, 81.09132871744148)
1 -- ('AMERICAN_Nationality', ('GREY_Color',), 83.58543897216273, 10.764882226980728, 0.9945723110437195, 83.13176320814767)
2 -- ('AMERICAN_Nationality', ('6_VehicleAge',), 83.58543897216273, 10.932762312633832, 0.988860516240292, 82.65434033218428)
3 -- ('Alloy_WheelType', ('MEDIUM SUV_Size',), 51.58201284796574, 10.95845824411135, 1.688940721840783, 87.1189620134438)
4 -- ('AMERICAN_Nationality', ('MEDIUM SUV_Size',), 83.58543897216273, 10.95845824411135, 1.0091714664183478, 84.35204001875879)
5 -- ('[462.0, 1165.6)_WarrantyCost_bin', ('OTHER ASIAN_Nationality',), 50.06423982869379, 11.02355460385439, 1.7165203392748303, 85.93628593628594)
6 -- ('AMERICAN_Nationality', ('2_VehicleAge',), 83.58543897216273, 11.594004282655245, 1.0482472113219743, 87.61820330969266)
7 -- ('AMERICAN_Nationality', ('CHRYSLER_Make',), 83.58543897216273, 12.150749464668094, 1.1

In [23]:
#RULES EXAMPLE
#INDEX: 
# 0 : 'AMERICAN_Nationality'
# 1 : ('[82449.4, 93538.6)_VehOdo_bin', 'AUTO_Transmission')
# 2 : 83.58515777572207 refers to 'Y' in report='YXlC'  that is relative head(head is 'AMERICAN_Nationality') item support as a percentage
# 3 : 20.27957652379484 refers to 'X' in report='YXlC'  that is X relative body(body is ('[82449.4, 93538.6)_VehOdo_bin', 'AUTO_Transmission')) set  support as a percentage
# 4 : 1.0436783505509324 refers to 'l' in report='YXlC'  that is lift value of a rule (confidence/prior)
# 5 : 87.23601959790506 refers to 'C' in report='YXlC'  that is rule confidence as a percentage
rules[4]

('AMERICAN_Nationality',
 ('MEDIUM SUV_Size',),
 83.58543897216273,
 10.95845824411135,
 1.0091714664183478,
 84.35204001875879)

In [25]:
#Code that filters all the association rules with a certain lift under
count = 0
for r in rules:
    if r[4]>1.02: #r[4] indeed refers to index 4 so as we have seen before index 4 refers to refers to 'l' in report='YXlC'  that is lift value of a rule (confidence/prior)
        print('%s -- %s' % (str(count), r))
        count = count + 1
print('Rules with high lift: {}'.format(count))

0 -- ('Alloy_WheelType', ('MEDIUM SUV_Size',), 51.58201284796574, 10.95845824411135, 1.688940721840783, 87.1189620134438)
1 -- ('[462.0, 1165.6)_WarrantyCost_bin', ('OTHER ASIAN_Nationality',), 50.06423982869379, 11.02355460385439, 1.7165203392748303, 85.93628593628594)
2 -- ('AMERICAN_Nationality', ('2_VehicleAge',), 83.58543897216273, 11.594004282655245, 1.0482472113219743, 87.61820330969266)
3 -- ('AMERICAN_Nationality', ('CHRYSLER_Make',), 83.58543897216273, 12.150749464668094, 1.1963806283688234, 100.0)
4 -- ('AMERICAN_Nationality', ('LARGE_Size',), 83.58543897216273, 12.195289079229124, 1.1590725093215024, 96.88158449220397)
5 -- ('AMERICAN_Nationality', ('[1869.2, 2572.8)_WarrantyCost_bin',), 83.58543897216273, 13.295074946466809, 1.1758782931577612, 98.28630331142894)
6 -- ('AMERICAN_Nationality', ('SE_Trim',), 83.58543897216273, 13.704496788008566, 1.092594608857828, 91.325)
7 -- ('[462.0, 1165.6)_WarrantyCost_bin', ('[49181.8, 60271.0)_VehOdo_bin',), 50.06423982869379, 13.946

In [31]:
#TASKS ON KAGGLE:
# 1) Frequent patterns extraction with different values of support and different types (i.e. frequent, close, maximal), (6 points)
# 2) Discussion of the most interesting frequent patterns and analyze how changes the number of patterns w.r.t. the min_sup parameter (7 points)

In [32]:
# s/a   sets/all   all     frequent item sets
for i in range(5,101,5):
    itemsets = apriori(baskets, supp=i, zmin=2, target='a') 
    print("Min support = {} \n Itemsets = {}".format(i,len(itemsets)))

Min support = 5 
 Itemsets = 442
Min support = 10 
 Itemsets = 100
Min support = 15 
 Itemsets = 29
Min support = 20 
 Itemsets = 12
Min support = 25 
 Itemsets = 8
Min support = 30 
 Itemsets = 5
Min support = 35 
 Itemsets = 3
Min support = 40 
 Itemsets = 2
Min support = 45 
 Itemsets = 1
Min support = 50 
 Itemsets = 0
Min support = 55 
 Itemsets = 0
Min support = 60 
 Itemsets = 0
Min support = 65 
 Itemsets = 0
Min support = 70 
 Itemsets = 0
Min support = 75 
 Itemsets = 0
Min support = 80 
 Itemsets = 0
Min support = 85 
 Itemsets = 0
Min support = 90 
 Itemsets = 0
Min support = 95 
 Itemsets = 0
Min support = 100 
 Itemsets = 0


In [26]:
# c     closed     closed  frequent item sets
for i in range(5,101,5):
    itemsets = apriori(baskets, supp=i, zmin=2, target='c')
    print("Min support = {} \n Itemsets = {}".format(i,len(itemsets)))

Min support = 5 
 Itemsets = 841
Min support = 10 
 Itemsets = 197
Min support = 15 
 Itemsets = 74
Min support = 20 
 Itemsets = 38
Min support = 25 
 Itemsets = 23
Min support = 30 
 Itemsets = 14
Min support = 35 
 Itemsets = 5
Min support = 40 
 Itemsets = 3
Min support = 45 
 Itemsets = 2
Min support = 50 
 Itemsets = 0
Min support = 55 
 Itemsets = 0
Min support = 60 
 Itemsets = 0
Min support = 65 
 Itemsets = 0
Min support = 70 
 Itemsets = 0
Min support = 75 
 Itemsets = 0
Min support = 80 
 Itemsets = 0
Min support = 85 
 Itemsets = 0
Min support = 90 
 Itemsets = 0
Min support = 95 
 Itemsets = 0
Min support = 100 
 Itemsets = 0


In [27]:
#m     maximal    maximal frequent item sets
for i in range(5,101,5):
    itemsets = apriori(baskets, supp=i, zmin=2, target='m') 
    print("Min support = {} \n Itemsets = {}".format(i,len(itemsets)))

Min support = 5 
 Itemsets = 358
Min support = 10 
 Itemsets = 98
Min support = 15 
 Itemsets = 42
Min support = 20 
 Itemsets = 22
Min support = 25 
 Itemsets = 16
Min support = 30 
 Itemsets = 14
Min support = 35 
 Itemsets = 5
Min support = 40 
 Itemsets = 3
Min support = 45 
 Itemsets = 2
Min support = 50 
 Itemsets = 0
Min support = 55 
 Itemsets = 0
Min support = 60 
 Itemsets = 0
Min support = 65 
 Itemsets = 0
Min support = 70 
 Itemsets = 0
Min support = 75 
 Itemsets = 0
Min support = 80 
 Itemsets = 0
Min support = 85 
 Itemsets = 0
Min support = 90 
 Itemsets = 0
Min support = 95 
 Itemsets = 0
Min support = 100 
 Itemsets = 0


In [28]:
#TASK ON KAGGLE:
# 3) Association rules extraction with different values of confidence (6 points)
# 4) Discussion of the most interesting rules and analyze how changes the number of rules w.r.t. the min_conf parameter, histogram of rules' confidence and lift (7 points)

In [29]:
#EXTRACT RULES
for i in range(5,101,5):
    for j in range(5,101,5):
        rules = apriori(baskets, supp=i, zmin=2, target='r', conf=j, report='YXlC') 
        print("Support = {} \nConfidence = {} \nRules = {}".format(i,j,len(rules)))
        #Code that filters all the association rules with a certain lift under
        count = 0
        for r in rules:
            if r[4]>1.4: #r[4] indeed refers to index 4 so as we have seen before index 4 refers to refers to 'l' in report='YXlC'  that is lift value of a rule (confidence/prior)
                count = count + 1
        print('Rules with high lift: {}'.format(count))
        print('\n')

Support = 5 
Confidence = 5 
Rules = 40127
Rules with high lift: 8025


Support = 5 
Confidence = 10 
Rules = 27738
Rules with high lift: 5061


Support = 5 
Confidence = 15 
Rules = 19989
Rules with high lift: 4009


Support = 5 
Confidence = 20 
Rules = 13745
Rules with high lift: 2975


Support = 5 
Confidence = 25 
Rules = 8906
Rules with high lift: 1998


Support = 5 
Confidence = 30 
Rules = 6569
Rules with high lift: 1395


Support = 5 
Confidence = 35 
Rules = 5300
Rules with high lift: 955


Support = 5 
Confidence = 40 
Rules = 4451
Rules with high lift: 754


Support = 5 
Confidence = 45 
Rules = 3692
Rules with high lift: 644


Support = 5 
Confidence = 50 
Rules = 3049
Rules with high lift: 565


Support = 5 
Confidence = 55 
Rules = 2392
Rules with high lift: 476


Support = 5 
Confidence = 60 
Rules = 1578
Rules with high lift: 405


Support = 5 
Confidence = 65 
Rules = 1169
Rules with high lift: 321


Support = 5 
Confidence = 70 
Rules = 869
Rules with high lift: 252


In [30]:
#HERE WE CAN SEE THE RULES THAT WE WANT
rules = apriori(baskets, supp=20, zmin=2, target='r', conf=70, 
                report='YXlC') 
print('Number of rules:', len(rules))
#Code that filters all the association rules with a certain lift under
c = 0
for r in rules:
        if r[4]>1: #r[4] indeed refers to index 4 so as we have seen before index 4 refers to refers to 'l' in report='YXlC'  that is lift value of a rule (confidence/prior)
            c += 1
            print('%s -- %s' % (str(c), r))
print('Rules with high lift: {}'.format(c))

Number of rules: 41
1 -- ('AMERICAN_Nationality', ('[82449.4, 93538.6)_VehOdo_bin',), 83.58543897216273, 20.86338329764454, 1.0366536473582555, 86.64915017653337)
2 -- ('AMERICAN_Nationality', ('CHEVROLET_Make',), 83.58543897216273, 23.70877944325482, 1.1963806283688234, 100.0)
3 -- ('AMERICAN_Nationality', ('OTHER_Auction',), 83.58543897216273, 23.97087794432548, 1.033933462364338, 86.4217823197313)
4 -- ('AMERICAN_Nationality', ('[71360.2, 82449.4)_VehOdo_bin',), 83.58543897216273, 30.425695931477513, 1.0068970008928084, 84.16192781937953)
5 -- ('AMERICAN_Nationality', ('[1165.6, 1869.2)_WarrantyCost_bin', 'Alloy_WheelType'), 83.58543897216273, 21.13747323340471, 1.1034936325038964, 92.23599967582462)
6 -- ('AMERICAN_Nationality', ('[1165.6, 1869.2)_WarrantyCost_bin', 'MANHEIM_Auction'), 83.58543897216273, 20.51220556745182, 1.1239423491331964, 93.94521463170203)
7 -- ('AMERICAN_Nationality', ('[1165.6, 1869.2)_WarrantyCost_bin',), 83.58543897216273, 34.922483940042824, 1.12771755885

In [31]:
# count how many rows for each attribute to make sure i haven't cut anything
for i in df_new:
    conta = 0
    for j in df_new.iterrows():
        conta += 1
    print(i, " = ", conta)

Auction  =  58375
VehicleAge  =  58375
Make  =  58375
Trim  =  58375
Color  =  58375
WheelType  =  58375
Nationality  =  58375
Size  =  58375
VNST  =  58375
VehBCost_bin  =  58375
VehOdo_bin  =  58375
WarrantyCost_bin  =  58375
